In [13]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
base_dir = '/content/drive/MyDrive/flowers'
print(f"Updated base_dir to: {base_dir}")

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import tensorflow
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense,Flatten,Dropout,Activation

import os

base_dir = '/content/drive/MyDrive/flowers'
img_size = 224
batch_size = 64

# Check if the base directory exists
if not os.path.exists(base_dir):
    print(f"Error: The directory '{base_dir}' does not exist. Please ensure Google Drive is mounted and the path is correct.")
else:
    # create a data Augmentator
    train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,validation_split=0.2)
    test_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

    # Creating the datasets

    train_datagen = train_datagen.flow_from_directory(base_dir,
                                                    target_size=(img_size,img_size),
                                                    subset='training',
                                                    batch_size=batch_size
                                                    )

    test_datagen = test_datagen.flow_from_directory(base_dir,
                                                    target_size=(img_size,img_size),
                                                    subset='validation',
                                                    batch_size=batch_size
                                                    )

    #Model Dveelopment

    model = Sequential()
    model.add(Conv2D(filters=64,kernel_size=(5,5),padding='same',activation='relu',input_shape=(224,224,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=32,kernel_size=(5,5),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=16,kernel_size=(5,5),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=8,kernel_size=(5,5),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(5,activation='softmax')) # Changed activation to softmax

    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

    model.fit(train_datagen,epochs=5,validation_data=test_datagen)


In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

In [ ]:
class_names = list(train_datagen.class_indices.keys())

def predict_image(img):
    # Resize the image to the model's expected input size
    img = img.resize((img_size, img_size))
    # Convert the image to a NumPy array
    img_array = np.array(img)
    # Normalize pixel values to the range [0, 1]
    img_array = img_array / 255.0
    # Add a batch dimension (e.g., from (224, 224, 3) to (1, 224, 224, 3))
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    predictions = model.predict(img_array)
    # Get the index of the predicted class
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    # Get the class name
    predicted_class_name = class_names[predicted_class_index]

    return predicted_class_name

print("Predict function 'predict_image' defined successfully.")
print(f"Class names extracted from train_datagen: {class_names}")

In [ ]:
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Textbox(label="Predicted Class"),
    title="Flower Image Classifier",
    description="Upload an image of a flower to get its predicted class (daisy, lily, rose, sunflower, or tulips)."
)

iface.launch(debug=True, share=True)
print("Gradio interface launched successfully.")